In [1]:
pip install opencv-python


   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/39.5 MB 487.6 kB/s eta 0:01:21
   ---------------------------------------- 0.1/39.5 MB 726.2 kB/s eta 0:00:55
   ---------------------------------------- 0.2/39.5 MB 871.5 kB/s eta 0:00:46
   ---------------------------------------- 0.2/39.5 MB 981.9 kB/s eta 0:00:40
   ---------------------------------------- 0.3/39.5 MB 1.0 MB/s eta 0:00:38
   ---------------------------------------- 0.4/39.5 MB 1.1 MB/s eta 0:00:36
   ---------------------------------------- 0.5/39.5 MB 1.2 MB/s eta 0:00:32
    --------------------------------------- 0.5/39.5 MB 1.3 MB/s eta 0:00:31
    --------------------------------------- 0.6/39.5 MB 1.3 MB/s eta 0:00:31
    --------------------------------------- 0.7/39.5 MB 1.3 MB/s eta 0:00:30
    --------------------------------------- 0.8/39.5 MB 1.4 MB/s eta 0:00:28
    -


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
pip install mediapipe


  Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached attrs-24.3.0-py3-none-any.whl.metadata (11 kB)
  Using cached jax-0.5.0-py3-none-any.whl.metadata (22 kB)
  Using cached jaxlib-0.5.0-cp312-cp312-win_amd64.whl.metadata (1.0 kB)
  Using cached matplotlib-3.10.0-cp312-cp312-win_amd64.whl.metadata (11 kB)
  Using cached opencv_contrib_python-4.11.0.86-cp37-abi3-win_amd64.whl.metadata (20 kB)
  Using cached sounddevice-0.5.1-py3-none-win_amd64.whl.metadata (1.4 kB)
  Using cached cffi-1.17.1-cp312-cp312-win_amd64.whl.metadata (1.6 kB)
  Using cached contourpy-1.3.1-cp312-cp312-win_amd64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.55.4-cp312-cp312-win_amd64.whl.metadata (169 kB)
  Using cached kiwisolver-1.4.8-cp312-cp312-win_amd64.whl.metadata (6.3 kB)
Using cached mediapipe-0.10.20-cp312-cp312-win_amd64.whl (51.0 MB)
Using cached attrs-24.3.0-py3-none-any.whl (63 kB)
Using cached

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import os
import cv2
import numpy as np
import mediapipe as mp
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
mp_drawing = mp.solutions.drawing_utils
hands = mp_hands.Hands(static_image_mode=True, max_num_hands=1, min_detection_confidence=0.5)


In [4]:
# Dataset directory
DATASET_DIR = "SignImage48x48"

In [5]:
def extract_hand_landmarks(image):
    """Extracts hand landmarks from an image using MediaPipe."""
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image_rgb)
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            landmarks = []
            for lm in hand_landmarks.landmark:
                landmarks.append([lm.x, lm.y, lm.z])
            return np.array(landmarks).flatten()
    return None

In [8]:
def load_data():
    """Loads the dataset, extracts hand landmarks, and prepares training data."""
    X, y = [], []
    for label, sign_class in enumerate(CLASSES):
        class_dir = os.path.join(DATASET_DIR, sign_class)
        if not os.path.exists(class_dir):
            continue
        for file_name in os.listdir(class_dir):
            file_path = os.path.join(class_dir, file_name)
            image = cv2.imread(file_path)
            if image is not None:
                landmarks = extract_hand_landmarks(image)
                if landmarks is not None:
                    X.append(landmarks)
                    y.append(label)

    return np.array(X), np.array(y)


In [11]:
# Load dataset
print("Loading data...")
CLASSES = [chr(i) for i in range(65, 91)]  # A-Z
IMAGE_SIZE = 128


Loading data...


In [13]:
# Load dataset
print("Loading data...")
X, y = load_data()

# Check if data is loaded correctly
if X.size == 0 or y.size == 0:
    raise ValueError("Dataset is empty or not loaded correctly. Ensure the dataset directory is set up properly.")

# Preprocess data
print("Preprocessing data...")
X = np.array(X, dtype=np.float32)
y = to_categorical(y, num_classes=len(CLASSES))

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build CNN model
model = Sequential([
    Dense(128, activation='relu', input_shape=(X.shape[1],)),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(len(CLASSES), activation='softmax')
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

# Train the model
checkpoint = ModelCheckpoint("sign_language_model.keras", save_best_only=True, monitor="val_loss", mode="min")

print("Training the model...")
history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=20,
    batch_size=32,
    callbacks=[checkpoint]
)

Loading data...
Preprocessing data...


c:\Users\USER\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │         8,192 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 26)             │         1,690 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 18,138 (70.85 KB)

 Trainable params: 18,138 (70.85 KB)

 Non-trainable params: 0 (0.00 B)

Training the model...
Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 2s 110ms/step - accuracy: 0.0565 - loss: 3.2167 - val_accuracy: 0.0000e+00 - val_loss: 3.1733
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step - accuracy: 0.1142 - loss: 3.1559 - val_accuracy: 0.2400 - val_loss: 3.0826
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - accuracy: 0.1831 - loss: 3.0521 - val_accuracy: 0.2800 - val_loss: 2.9956
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.2225 - loss: 2.9402 - val_accuracy: 0.2800 - val_loss: 2.9064
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step - accuracy: 0.2165 - loss: 2.7993 - val_accuracy: 0.3200 - val_loss: 2.8376
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step - accuracy: 0.2728 - loss: 2.8106 - val_accuracy: 0.3200 - val_loss: 2.7713
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.2049 - loss: 2.7922 - val_accuracy: 0.3600 - val_loss: 2.7150
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step - accuracy: 0.2333 - loss: 2.7333 - val_accuracy:

In [14]:
# Save the final model
model.save("sign_language_model_final.keras", save_format='keras')
print("Model training complete and saved.")

Model training complete and saved.
